In [ ]:
%load_ext autoreload
%autoreload 2
from utils.load import *
from utils.visualisation import *
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# cfg_file = "config.yaml"

# config = read_config(cfg_file)


Load Radar Dataset

In [ ]:
from utils.load import load_radar_dataset

radar_df = load_radar_dataset(folder_name='test')

print(radar_df.iloc[0]["data"].shape)

In [ ]:
#cml_df = load_cml_dataset(dataset_name="CML_data_processed_2025.nc")
# gauge_df = load_raingauge_dataset(dataset_name='station_locations.csv')

Load raingauge dataset

Filter formatted gauge dataframe to only include if at least N sensors have values greater than 0

In [ ]:
N = 0
raingauge_df = load_raingauge_dataset('rainfall_data.csv', N=0)

In [ ]:
print(raingauge_df)

# Filter for only stations whos coordinates we know

In [ ]:
station_dict = get_gauge_mappings()
filter_cols = [s for s in raingauge_df.columns if s in station_dict]
station_aligned_raingauge_df = raingauge_df[filter_cols]
print(station_aligned_raingauge_df)

# MERGE RADAR DATA AND RAIN GAUGE DATA

In [ ]:
merged_df = pd.merge(station_aligned_raingauge_df, radar_df, on='time_sgt', how='inner')

In [ ]:
print(merged_df.values)

ATTEMPT AT KRIGING BASELINE

In [ ]:
from pykrige.uk import UniversalKriging

row_data = merged_df.iloc[3].dropna()
station_names = list(row_data.index[:-4])

data = []

for s in station_names[1:]:
  lat, long = station_dict[s]
  data.append([long, lat, row_data[s]])

gauge_data = np.array(data)

gridx = np.arange(103.6, 104.1, 0.01)
gridy = np.arange(1.25, 1.47, 0.007)

#RADAR FOR USE IN EXTERNAL DRIFT

radar_grid = row_data['data']
bounds = row_data['bounds']
transform = row_data['transform']
x_min = bounds.left
y_max = bounds.top
pixel_width = transform[0]
pixel_height = -transform[4]

e_d = []
e_dx = []
e_dy = []

for row in range(radar_grid.shape[0]): 
    y = y_max - (row * pixel_height) + pixel_height / 2
    e_dy.append(y)

for col in range(radar_grid.shape[0]):

    # Calculate middle of cell
    x = x_min + (col * pixel_width) + pixel_width / 2
    e_dx.append(x)


e_dx = np.array(e_dx)
e_dy = np.array(e_dy)


KED = UniversalKriging(
    x=gauge_data[:, 0],
    y=gauge_data[:, 1],
    z=gauge_data[:, 2],
    variogram_model="linear",
    drift_terms=["external_Z"],
    external_drift=radar_grid,
    external_drift_x=e_dx,
    external_drift_y=e_dy,
)

UK = UniversalKriging(
    gauge_data[:, 0],
    gauge_data[:, 1],
    gauge_data[:, 2],
    variogram_model="linear",
    drift_terms=["regional_linear"],
)


z,ss = KED.execute("grid", gridx, gridy)
z2,ss2 = UK.execute("grid", gridx, gridy)
fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].imshow(z, origin='lower')
axes[1].imshow(z2, origin='lower')
plt.show()

print(z)

Plot raingauge and radar data on the same grid

# General plotting function (Hard coded for max 9 plots for now)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(23, 12))

bounds_singapore = {
  'left': 103.6,
  'right': 104.05,
  'top': 1.5,
  'bottom': 1.15
}

#iterrate through rows
for index, row in merged_df.head(4).iterrows():
  node_df = pandas_to_geodataframe(row)
  radar_input = row[['data', 'bounds', 'crs', 'transform']]
  visualise_gauge_grid(node_df=node_df, ax=ax[int(index / 2)][index % 2])
  visualise_radar_grid(data=radar_input, ax=ax[int(index / 2)][index % 2], zoom=bounds_singapore, scaling=None, alpha=0.5, legend=False)

plt.plot()

  

In [ ]:
print(len(merged_df))